In [ ]:
import torch
import torch.nn as nn

from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import datasets, transforms
import torchvision.models as models

from sklearn.metrics import accuracy_score

import numpy as np
import pandas as pd

import os
import random
from tqdm import tqdm
from PIL import Image

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

seed_everything(7)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
class Crop20(object):
    def __call__(self, image):
        if image.size == (1080, 1920):
            return image.crop((0,0,1080,1536))
        else:
            return image
        
class ConditionalResize:
    def __init__(self, target_size):
        self.target_size = target_size

    def __call__(self, img):
        if max(img.size) < self.target_size:
            return transforms.Resize(self.target_size, interpolation=transforms.InterpolationMode.BILINEAR)(img)
        return img

In [ ]:
model = models.efficientnet_v2_s()
num_ftrs = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_ftrs, 2)
model = model.to(device)

#모델 업로드 경로 지정
model_path = ''
model.load_state_dict(torch.load(model_path))
model.eval()
final_predictions = []

In [ ]:
target_size = 380
test_transform = transforms.Compose([
    Crop20(),
    ConditionalResize(target_size),
    transforms.CenterCrop(target_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

class CustomDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        image_name = self.df.iloc[idx, 0]
        image_path = f'/kaggle/input/image-classification-2024-spring/dataset/test/{image_name}'
        image = Image.open(image_path)

        if self.transform:
            image = self.transform(image)

        return image
    
test_df = pd.read_csv('/kaggle/input/image-classification-2024-spring/submission_sample.csv')

test_dataset = CustomDataset(test_df, transform=test_transform)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

with torch.no_grad():
    for images in tqdm(test_dataloader):
        images = images.to(device)
        outputs = model(images)
        preds = torch.sigmoid(outputs)
        _, predicted = torch.max(preds.data, 1)
        final_predictions += predicted.detach().cpu().numpy().tolist()

test_df['label'] = final_predictions
test_df.to_csv('submission_model3.csv', index=False)